# CSCI451 Project: UNHCR ML Challenge
Jamie Hackney, Mihir Singh, Jake Gilbert

In [15]:
# imports
import numpy as np
import pandas as pd
# imports for models
from sklearn.linear_model import LinearRegression
from sklearn import tree 
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor

# create models
LR = LinearRegression()
Svc = SVC()
Tree = DecisionTreeRegressor()
NN = MLPClassifier()

# load the data
df = pd.read_csv('/Users/mihirsingh/Documents/Middlebury/CSCI451/UNHCR-ml-challenge/data/combined_data.csv')
df_imp = df.copy()

# encode data that is not numerical
encoder = LabelEncoder()
encoder.fit(df_imp['Region'])
df_imp['Region'] = encoder.transform(df_imp['Region'])
encoder.fit(df_imp['District'])
df_imp['District'] = encoder.transform(df_imp['District'])
encoder.fit(df_imp['Month'])
df_imp['Month'] = encoder.transform(df_imp['Month'])


# turn string numbers into floats
features = ['CDI','Month','Year','NDVI','Rainfall','Water Price',
            'Conflict Fatalities','Conflict Incidents','Cholera Deaths',
            'Cholera Cases','Malaria','Measles','Cost Min Basket',
            'Goat Price','Goat to Cereal','Maize Price','Rice Price',
            'Sorghum Price','Wage Price','Wage to Cereal', 'Departures','Arrivals']

for feature in features:
    df_imp[feature] = df_imp[feature].apply(lambda x: x.replace(',', '') if isinstance(x, str) else x)
    
# impute data
# imp = SimpleImputer(missing_values=np.nan, strategy='median')
# array_imp = imp.fit_transform(df_imp)
# df_imp = pd.DataFrame(array_imp, columns=df.columns)

#L1 regression - Lasso

imp = IterativeImputer(max_iter=10, random_state=0)
array_imp = imp.fit_transform(df_imp)
df_imp = pd.DataFrame(array_imp, columns=df.columns)   
# df_imp = df_imp.dropna()


/var/folders/fv/_v7myjl904983sbb39qbkzw40000gn/T/ipykernel_57093/4157640453.py:25: DtypeWarning: Columns (17,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/mihirsingh/Documents/Middlebury/CSCI451/UNHCR-ml-challenge/data/combined_data.csv')
/Users/mihirsingh/anaconda3/envs/ml-0451/lib/python3.9/site-packages/sklearn/impute/_iterative.py:796: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [2]:

df_imp.head()

,Unnamed: 0,Region,District,CDI,Month,Year,NDVI,Rainfall,Water Price,Conflict Fatalities,...,Cost Min Basket,Goat Price,Goat to Cereal,Maize Price,Rice Price,Sorghum Price,Wage Price,Wage to Cereal,Departures,Arrivals
0,0.0,0.0,13.0,1.05,4.0,2018.0,0.168,4.342,4000.000000,0.0,...,1.090175e+06,2.977500e+05,57.000000,4000.000000,5200.000000,3200.000000,57400.000000,11.000000,6.000000,199.0
1,1.0,0.0,5.0,0.62,4.0,2018.0,0.132,6.065,8816.146932,0.0,...,3.221842e+06,1.034149e+06,96.505868,12674.026322,20302.198169,15778.178913,103823.463756,9.562667,415.870257,69.0
2,2.0,0.0,53.0,0.63,4.0,2018.0,0.047,7.915,15724.612735,0.0,...,1.674934e+06,3.000000e+05,50.000000,4729.703777,7000.000000,6000.000000,90000.000000,15.000000,253.000000,24.0
3,3.0,0.0,73.0,1.10,4.0,2018.0,0.052,8.121,6000.000000,0.0,...,1.160475e+06,4.100000e+05,59.000000,6000.000000,7000.000000,5500.000000,100000.000000,14.000000,-32.184029,68.0
4,4.0,17.0,41.0,0.78,4.0,2018.0,0.000,2.247,11000.000000,0.0,...,1.366665e+06,4.600000e+05,92.000000,6000.000000,5000.000000,5000.000000,70000.000000,14.000000,400.000000,663.0


In [3]:
# split data into training and testing data
columns = ['Region', 'District','CDI','Month','Year','NDVI','Rainfall','Water Price',
            'Conflict Fatalities','Conflict Incidents','Cholera Deaths',
            'Cholera Cases','Malaria','Measles','Cost Min Basket',
            'Goat Price','Goat to Cereal','Maize Price','Rice Price',
            'Sorghum Price','Wage Price','Wage to Cereal', 'Departures']
X_train, X_test, y_train, y_test = train_test_split(df_imp[columns], df_imp['Arrivals'], test_size=0.15)
print(X_test.shape)
print(y_test.shape)

(49329, 23)
(49329,)


With the data processed, we can now build some models.

In [4]:
def has_negative(values):
    for value in values:
        if value < 0:
            return True
    return False

In [5]:
import math

# duplicate data
LR_xtrain = X_train.copy()
LR_ytrain = y_train.copy()
LR_xtest = X_test.copy()
LR_ytest = y_test.copy()

# fit and predict the model
LR.fit(LR_xtrain, list(LR_ytrain))
LR_pred = LR.predict(LR_xtest)

# calculate R^2
print("R^2: ", LR.score(LR_xtest, list(LR_ytest)))

# # bin LR_pred and bin y_test
LR_pred = pd.DataFrame(LR_pred, columns=['Arrivals'])
LR_pred['bins'] = pd.cut(LR_pred['Arrivals'], bins=[float('-inf'), 1000, 5000, float('inf')], labels=[1,2,3], right = False)
ybins = pd.cut(y_test, bins=[float('-inf'), 1000, 5000, float('inf')], labels=[1,2,3], right = False)

# evaluate
def classification_accuracy(y_true, y_pred):
    if len(y_true) != len(y_pred):
        raise ValueError("Arrays must be of equal length")
    
    correct_predictions = np.sum(y_true == y_pred)
    total_predictions = len(y_true)
    accuracy = correct_predictions / total_predictions
    return accuracy

print("Accuracy: ", classification_accuracy(ybins, list(LR_pred['bins'])))    

R^2:  0.769176882742646
Accuracy:  0.8475541770560928


In [16]:
import math

# duplicate data
DT_xtrain = X_train.copy()
DT_ytrain = y_train.copy()
DT_xtest = X_test.copy()
DT_ytest = y_test.copy()

# fit and predict the model
Tree.fit(DT_xtrain, list(DT_ytrain))
DT_pred = Tree.predict(LR_xtest)

# calculate R^2
print("R^2: ", Tree.score(DT_xtest, list(DT_ytest)))

# # bin LR_pred and bin y_test
DT_pred = pd.DataFrame(DT_pred, columns=['Arrivals'])
DT_pred['bins'] = pd.cut(DT_pred['Arrivals'], bins=[float('-inf'), 1000, 5000, float('inf')], labels=[1,2,3], right = False)
ybins = pd.cut(y_test, bins=[float('-inf'), 1000, 5000, float('inf')], labels=[1,2,3], right = False)

# evaluate
print("Accuracy: ", classification_accuracy(ybins, list(DT_pred['bins'])))    

R^2:  0.9757119329988229
Accuracy:  0.9955604208477772


In [6]:
import math


# fitting models
LR.fit(X_train, list(y_train))

# predicting
LR_pred = LR.predict(X_test)


# evaluate
print("on test data: ", math.sqrt(mean_squared_error(list(y_test), LR_pred)))
print(f'rmse train: {math.sqrt(mean_squared_error(LR.predict(X_train), y_train))}')

# caluclate R^2
# r2_score(list(y_test), LR_pred)
LR.score(X_test, list(y_test))

on test data:  2162.092689427628
rmse train: 2139.1523560292435


0.769176882742646

Studies that try to explain human behavior generally have R^2 values lower than 50%

In [7]:
from sklearn.linear_model import Lasso

RF = Lasso()
RF.fit(X_train, y_train)
RF.score(X_test, y_test)
print(f'rmse: {math.sqrt(mean_squared_error(RF.predict(X_test), y_test))}')

rmse: 2161.836827131317


In [8]:
from sklearn.linear_model import ElasticNet
RF = ElasticNet()
RF.fit(X_train, y_train)
score = RF.score(X_test, y_test)
print(score)
print(f'rmse: {math.sqrt(mean_squared_error(RF.predict(X_test), y_test))}')

0.7591107848171775
rmse: 2208.7335892278193


In [9]:
from sklearn.tree import DecisionTreeRegressor

RF = DecisionTreeRegressor()
RF.fit(X_train, y_train)
score = RF.score(X_test, y_test)
print(score)
print(f'rmse train: {math.sqrt(mean_squared_error(RF.predict(X_train), y_train))}')
print(f'rmse: {math.sqrt(mean_squared_error(RF.predict(X_test), y_test))}')
print("Accuracy: ", classification_accuracy(ybins, list(LR_pred['bins'])))    

0.972717525059556
rmse train: 5.29743001307986e-07
rmse: 743.3212351532419


In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectKBest, f_regression
sel = SelectKBest(f_regression, k=6)
X_new = sel.fit_transform(X_train, y_train)
cols_idxs = sel.get_support(indices=True)
print(cols_idxs)
features_df_new = X_train.iloc[:,cols_idxs]

print(features_df_new.columns)

[ 7  9 11 13 15 22]
Index(['Water Price', 'Conflict Incidents', 'Cholera Cases', 'Measles',
       'Goat Price', 'Departures'],
      dtype='object')


In [11]:

RF = RandomForestRegressor(max_depth=11)
RF.fit(X_train, y_train)
print(RF.score(X_test, y_test))
print(f'rmse train: {math.sqrt(mean_squared_error(RF.predict(X_train), y_train))}')
print(f'rmse: {math.sqrt(mean_squared_error(RF.predict(X_test), y_test))}')

0.971442904510204
rmse train: 628.8751713058189
rmse: 760.4867894810534
